# 6.1 Batch Normalization (BN)
本节旨在介绍[Batch Normalization](https://zhuanlan.zhihu.com/p/34879333)及其[实现方法](https://zhuanlan.zhihu.com/p/87117010)，以及[BN对于模型性能的影响](https://mp.weixin.qq.com/s/FFLQBocTZGqnyN79JbSYcQ)。

## 6.1.1 Batch Normalization原理
随着网络深度的增加，每层特征的分布会逐渐向激活函数输出区间的上下两端靠近（激活函数饱和区间），致使梯度消失，与此同时，每层特征的分布也不再满足i.i.d.假设。

<p align=center>
<img src="./fig/6-15.png" width=300>
</p>

2015年，Google提出了Batch Normalization (BN)来试图解决这一问题

BN通过批标准化，希望每层特征的分布重新拉回到标准正态分布，1.使得每层网络的输入能尽量保持相同分布以去除Internal Covariate Shifts (ICS); 2.使特征能够处于激活函数的梯度对输入较为敏感的区间，从而使得梯度变大，避免梯度消失的同时加快模型收敛速度。（Relu, resnet等方法都注意到了这个问题）

BN通过保留通道channel的维度，对每个batch中的特征按维度做标准化处理（图片作为样本的话就是按照HW整张图做标准化）：

<p align=center>
<img src="./fig/6-0.png" width=300>
</p>

为方便理解，这里举一个可视化例子：

<p align=center>
<img src="./fig/6-1.png" width=500>
</p>

得到各维标准化后的特征数据

<p align=center>
<img src="./fig/6-2.png" width=500>
</p>

虽然这种归一化操作缓解了ICS问题，但却导致了数据表达能力的缺失，因为改变了原有数据的信息表达，使得原网络学习到的参数信息丢失。因此，BN引入两个可学习的参数$\gamma, \beta$来恢复数据本身的表达能力，对标准化后的数据进行线性变换
$$\tilde{Z}_j=\gamma_j \hat{Z}_j+\beta_j$$
特别地，当$\gamma^2=\sigma^2$且$\beta=\mu$时，该线性变换可以恢复原有数据的分布，实现等价变换。

因此，通过引入可学习的 $\gamma, \beta$以及该线性变换，我们一定程度上保留了输入数据的表达能力，使得网络可以根据loss调整对特征分布的变换，以得到满意的效果。

在测试阶段，BN的可学习参数将被固定，并保留了每个batch的训练数据在网络每一层的$\mu_{batch}$和$\sigma^2_{batch}$，在此基础上无偏估计整体数据分布的均值与协方差用于测试。测试数据将使用$\mu_{test}$和$\sigma^2_{test}$来标准化数据，即：

<p align=center>
<img src="./fig/6-3.png" width=500>
</p>

## 6.1.2 BN的优势与缺陷
在此将BN的优势总结为4点，详情可参考[知乎-天雨粟](https://zhuanlan.zhihu.com/p/34879333):
+ BN使得网络中每层输入数据的分布相对稳定，加速模型学习速度
+ BN使得模型对网络中设置的学习率不那么敏感，简化调参过程，使得网络学习更加稳定
+ BN允许网络使用饱和性激活函数（例如sigmoid，tanh等），缓解梯度消失问题
+ BN具有一定的正则化效果（Batch对总体的抽样为网络学习增加了随机噪声，类似于drop out关闭神经元给训练带来噪声，提升模型泛化性）

BN的缺陷包含：
+ 对batch size比较敏感，较小的batch size会带来很大的采样变差，从而使得模型性能明显恶化；
+ 在递归结构的序列模型中（如RNN），水平方向做BN（沿时间步而非网络序列步）效果很差，因为RNN的递归网络结构可以适应动态可变的输入序列长度，而这样会导致BN并不能正确的使用[（因为BN是对每个时间步的所有样本做标准化，要维护每个时间步对应的均值与方差，而训练与测试实例有长有短，长序列后面的神经元做BN估计会有很大偏差）](https://stackoverflow.com/questions/45493384/is-it-normal-to-use-batch-normalization-in-rnn-lstm)

## 6.1.3 BN的Torch实现与应用
首先对torch中的nn.BatchNorm1d及其用法进行介绍，而后通过一个例子展现BN在网络设计中的应用。

其中，Batch Normalization Transform的计算方式如下图所示，Affine Transform即对normalized的数据进行可学习的线性变换以保留输入数据的表达能力。

<p align=center>
<img src="./fig/6-4.png" width=600>
</p>

### 6.1.3.1 nn.BatchNorm1d/2d/3d介绍
Torch中的nn包已经对不同的数据处理格式实现了BN操作，通用的调用格式为：
```
nn.BatchNorm'X'd(num_features, eps = 1e-5, momentum = 0.1, affine = True, track_running_stats = True)
```
'X' = 1,2,3；

`num_features`：每次归一化的特征数量；

`eps`：分母修正项$\epsilon$；

`momentum`：动量值，用于指数加权平均估计当前的均值与方差（类似于Adam的那种动量）；

`affine`：是否需要可训练的affine transform；

`track_running_stats`：True为训练状态，False为测试状态；


<p align=center>
<img src="./fig/6-6.png" width=800>
</p>

对于`nn.BatchNorm1d()`而言，其处理方式为：

<p align=center>
<img src="./fig/6-5.png" width=800>
</p>

对于`nn.BatchNorm2d()`而言，其处理方式为：

<p align=center>
<img src="./fig/6-7.png" width=800>
</p>

对于`nn.BatchNorm3d()`而言，其处理方式为：

<p align=center>
<img src="./fig/6-8.png" width=800>
</p>

### 6.1.3.2 BN在网络设计中的应用
本节通过LeNet为例，实现一种带有BN的网络（BN通常放在激活函数 前或后，需要根据实验结果来确定）。关于实验结果对比可以参考[深入理解BN](https://zhuanlan.zhihu.com/p/87117010)

In [1]:
import torch
from torch import nn

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        
        # Convolution sequential
        self.conv = nn.Sequential(
            nn.Conv2d(1, 20, kernel_size=5), # input channel:1, output channel:20
            nn.MaxPool2d(2),
            nn.BatchNorm2d(20), # corresponding to the number of channels
            nn.ReLU(),
            nn.Conv2d(20, 50, kernel_size=5),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(50),
            nn.ReLU()
        )

        # Dense sequential
        self.dense = nn.Sequential(
            nn.Linear(800, 500),
            nn.ReLU(),
            nn.Linear(500, 10)
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = torch.flatten(x, start_dim = 1)
        x = self.dense(x)
        return x